# 使い方・このノートブックについて

- Google Colaboratoryで使用することを想定しています。
- GPUインスタンスで実行してください
- 実行前にdatasetフォルダ内に、データセットへのリンクとしてtrainとtestという名前でシンボリックリンクを作成してください



# パラメータ設定


In [0]:
#@title datasetとoutputフォルダの設定
#@markdown デフォルト値は声優統計コーパス音声をgithubからcloneしてtrain/testに使用する。Googleドライブの「My Drive/Colab Notebooks/VoicePix2Pix」というフォルダに結果データを保存する。
#@markdown 自分のGoogleドライブ内のdatasetを使用したいときは、Googleドライブをマウントしてから左ペインのファイルタブからマウント先フォルダを探すとよい
dataset_train = '/content/voice-statistics.github.com/assets/data/fujitou_normal'  #@param {type: "string"}
dataset_test = '/content/voice-statistics.github.com/assets/data/tsuchiya_angry'  #@param {type: "string"}
output = '/content/drive/My Drive/Colab Notebooks/VoicePix2Pix'  #@param {type: "string"}
#@markdown ---

import os
# dataset_train = os.path.join('/content/drive/',dataset_train) + "/"
# dataset_test = os.path.join('/content/drive/',dataset_test) + "/"
# output = os.path.join('/content/drive/',output) + "/"

os.environ['path_train'] = dataset_train
os.environ['path_test'] = dataset_test
os.environ['path_output'] = output


# 環境構築

## インストール


In [1]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

# Chainer/CuPy Installer for Google Colaboratory
# https://github.com/kmaehashi/chainer-colab
!curl https://colab.chainer.org/install | sh -

#world4py インストール
%cd /content
!git clone --depth 1 https://github.com/yamachu/WORLD4py
%cd /content/WORLD4py
!python setup.py install

# VoicePix2Pixをclone
%cd /content
!git clone --depth 1 https://github.com/lilacs2039/VoicePix2Pix
%cd /content/VoicePix2Pix
!pip install -r requirements.txt


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1379  100  1379    0     0   5126      0 --:--:-- --:--:-- --:--:--  5126
+ apt -y -q install cuda-libraries-dev-9-2
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  cuda-cublas-dev-9-2 cuda-cufft-dev-9-2 cuda-cur

## 音声データをダウンロード、解凍、リンク作成



In [2]:
# 声優統計コーパスをクローン
%cd /content
!git clone --depth 1 https://github.com/voice-statistics/voice-statistics.github.com
# *.tar.gzで圧縮された音声データを解凍して、圧縮ファイルを削除
%cd /content/voice-statistics.github.com/assets/data
!ls *.gz |xargs -n1 tar -xzf
!ls *.gz |xargs -n1 rm
!ls


/content
Cloning into 'voice-statistics.github.com'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 87 (delta 2), reused 71 (delta 2), pack-reused 0
Unpacking objects: 100% (87/87), done.
Checking out files: 100% (71/71), done.
/content/voice-statistics.github.com/assets/data
fujitou_angry  fujitou_normal  tsuchiya_happy	uemura_angry  uemura_normal
fujitou_happy  tsuchiya_angry  tsuchiya_normal	uemura_happy


In [6]:
# outputフォルダをGoogleドライブに作成
!mkdir -p "$path_output"

# シンボリックリンクを作成
%cd /content/VoicePix2Pix
!mkdir dataset
!mkdir output
!rm /content/VoicePix2Pix/dataset/train
!ln -s "$path_train" /content/VoicePix2Pix/dataset/train
!rm /content/VoicePix2Pix/dataset/test
!ln -s "$path_test" /content/VoicePix2Pix/dataset/test
!rm /content/VoicePix2Pix/output
!ln -s "$path_output" /content/VoicePix2Pix/output

%cd /content/VoicePix2Pix


/content/VoicePix2Pix
mkdir: cannot create directory ‘dataset’: File exists
mkdir: cannot create directory ‘output’: File exists
rm: cannot remove '/content/VoicePix2Pix/dataset/train': No such file or directory
rm: cannot remove '/content/VoicePix2Pix/dataset/test': No such file or directory
rm: cannot remove '/content/VoicePix2Pix/output': Is a directory
/content/VoicePix2Pix


# 学習開始

In [0]:
# GPUありで学習
%cd /content/VoicePix2Pix
!python train.py -g 0 -i ./dataset --out ./output --snapshot_interval 1000 --shared_mem 64000000

# CPUで学習する場合
#!python train.py -g -1 -i ./dataset --out ./output --snapshot_interval 1000 --shared_mem 64000000



/content/VoicePix2Pix
GPU: 0
# Minibatch-size: 1
# epoch: 200

search dataset paths
    from: ./dataset/train
load dataset paths done
search dataset paths
    from: ./dataset/test
load dataset paths done
     total [..................................................]  0.05%
this epoch [#####.............................................] 10.00%
        10 iter, 0 epoch / 200 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [..................................................]  0.10%
this epoch [##########........................................] 20.00%
        20 iter, 0 epoch / 200 epochs
   0.41654 iters/sec. Estimated time to finish: 13:19:26.907220.
     total [..................................................]  0.15%
this epoch [###############...................................] 30.00%
        30 iter, 0 epoch / 200 epochs
   0.46011 iters/sec. Estimated time to finish: 12:03:22.415609.
     total [..................................................]  0.20

In [4]:
!nvidia-smi


data_loader.py	     LICENSE  README.md		train.py     util.py
dataset		     net.py   requirements.txt	updater.py
image_visualizer.py  output   test		Usage.ipynb
